In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
371,7.7,0.29,0.48,2.3,0.049,36.0,178.0,0.99310,3.17,0.64,10.6,6
1967,8.0,0.23,0.35,9.2,0.044,53.0,186.0,0.99700,3.09,0.56,9.5,7
110,6.5,0.17,0.54,8.5,0.082,64.0,163.0,0.99590,2.89,0.39,8.8,6
1765,6.9,0.29,0.23,8.6,0.056,56.0,215.0,0.99670,3.17,0.44,8.8,5
1616,6.9,0.34,0.49,7.3,0.045,61.0,206.0,0.99570,3.09,0.40,9.0,6
3455,7.3,0.23,0.24,0.9,0.031,29.0,86.0,0.98926,2.90,0.38,12.2,6
1894,6.8,0.35,0.44,6.5,0.056,31.0,161.0,0.99520,3.14,0.44,9.5,5
3067,7.3,0.28,0.42,1.2,0.033,29.0,142.0,0.99205,3.17,0.43,10.7,4
3841,6.6,0.21,0.50,8.7,0.036,41.0,191.0,0.99294,2.96,0.56,11.0,6
1744,6.6,0.22,0.58,1.1,0.133,52.0,136.0,0.99320,3.10,0.30,9.1,5


In [3]:
remote_server_uri = "http://0.0.0.0:5000" 
mlflow.set_tracking_uri(remote_server_uri)

In [4]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [6]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/08/29 22:37:08 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/826729787346648927', creation_time=1724951228933, experiment_id='826729787346648927', last_update_time=1724951228933, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [7]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [8]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: mlflow-artifacts:/826729787346648927/a01183d302e54020bb0d238e490fe854/artifacts


2024/08/29 22:38:35 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/29 22:38:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run trusting-zebra-901 at: http://0.0.0.0:5000/#/experiments/826729787346648927/runs/a01183d302e54020bb0d238e490fe854.
2024/08/29 22:38:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5000/#/experiments/826729787346648927.


In [9]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: mlflow-artifacts:/826729787346648927/1f1602f7a9614f99a9360cf72e22220b/artifacts


2024/08/29 22:38:44 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/29 22:38:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-ram-120 at: http://0.0.0.0:5000/#/experiments/826729787346648927/runs/1f1602f7a9614f99a9360cf72e22220b.
2024/08/29 22:38:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5000/#/experiments/826729787346648927.


In [10]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlflow-artifacts:/826729787346648927/c7e18a7125ce45dcbcb21fef8fae79f4/artifacts


2024/08/29 22:38:53 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/29 22:38:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run amazing-shoat-321 at: http://0.0.0.0:5000/#/experiments/826729787346648927/runs/c7e18a7125ce45dcbcb21fef8fae79f4.
2024/08/29 22:38:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5000/#/experiments/826729787346648927.
